Install dependencies

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install peft

Import dependencies

In [1]:
import evaluate
import torch
from datasets import load_dataset
from peft import get_peft_model, LoraConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer

Load the Hugging Face dataset 'emotion'

In [ ]:
dataset = load_dataset("emotion")

Explore the dataset, consisting of train, validation and test dataset

In [3]:
display(dataset.shape)

train = dataset['train']
emotion_labels = set()

for labels in train:
  emotion_labels.add(labels['label'])

print("Labels in the dataset: ", emotion_labels)

{'train': (16000, 2), 'validation': (2000, 2), 'test': (2000, 2)}

Labels in the dataset:  {0, 1, 2, 3, 4, 5}


From https://huggingface.co/datasets/dair-ai/emotion dataset documentation, we know what the labels represent:

0: sadness, 1: joy, 2: love, 3: anger, 4: fear, 5: surprise

Another way to approach this is by checking the features

In [4]:
dataset['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

Let's know look at an example:

In [5]:
dataset['train'][9]

{'text': 'i feel romantic too', 'label': 2}

Load DistilBERT transformer model

In [ ]:
model_checkpoint = 'distilbert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=6)

Check the model properties

In [7]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

Import the tokenizer and create a function to tokenize the whole dataset

In [8]:
def tokenize(rows):
    return tokenizer(rows['text'], padding="max_length", truncation=True)

model_tokenizer = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_tokenizer)

dataset.set_format(type=None)

tokenized_datasets = dataset.map(tokenize, batched=True)

Tokenize our example

In [9]:
print("Example: ", dataset['train'][9]['text'])
tokenized_example = tokenizer(dataset['train'][9]['text'])
print("Tokenized example: ", tokenized_example)

Example:  i feel romantic too
Tokenized example:  {'input_ids': [101, 1045, 2514, 6298, 2205, 102], 'attention_mask': [1, 1, 1, 1, 1, 1]}


Initiate a data collator with padding

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Define the evaluation metrics

In [11]:
import evaluate
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

Define the PeftModel parameters

In [17]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])

Check the parameters of the initiated PeftModel



In [18]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False)

Create a PeftModel from the configuration and base model

In [19]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 632,070 || all params: 67,590,156 || trainable%: 0.9351509708011326


Define the training arguments

In [20]:
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-emotion-classification",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

Define the trainer and start the training

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.591200,0.507732,0.831500
2,0.486100,0.437479,0.870500
3,0.422300,0.401381,0.888000
4,0.399300,0.378572,0.899500
5,0.369500,0.376783,0.898500


TrainOutput(global_step=20000, training_loss=0.5096245742797851, metrics={'train_runtime': 3163.0477, 'train_samples_per_second': 25.292, 'train_steps_per_second': 6.323, 'total_flos': 1.075348537344e+16, 'train_loss': 0.5096245742797851, 'epoch': 5.0})

Finally, evaluate your model on a completely new test dataset

In [22]:
test_dataset = tokenized_datasets["test"]

results = trainer.evaluate(eval_dataset=test_dataset)

print(results)

{'eval_loss': 0.3749876618385315, 'eval_accuracy': 0.89, 'eval_runtime': 34.7144, 'eval_samples_per_second': 57.613, 'eval_steps_per_second': 14.403, 'epoch': 5.0}
